# 마르코프 체인

In [1]:
import requests
import codecs
from bs4 import BeautifulSoup
from konlpy.tag import Mecab
import xmltodict
import os, json, random

In [17]:
# 마르코프 체인의 딕셔너리 생성
def make_dic(words):
    tmp = ["@"]
    dic = {}
    for word in words:
        tmp.append(word)
        if len(tmp) < 3:
            continue
        if len(tmp) > 3:
            tmp = tmp[1:]
        set_word3(dic, tmp)
        if word == ".":
            tmp = ["@"]
            continue
    return dic

# 딕셔너리에 데이터를 등록
def set_word3(dic, s3):
    w1, w2, w3 = s3
    if not w1 in dic: dic[w1] = {}
    if not w2 in dic[w1]: dic[w1][w2] = {}
    if not w3 in dic[w1][w2]: dic[w1][w2][w3] = 0
    dic[w1][w2][w3] += 1

# 문장을 생성
def make_sentence(dic):
    ret = []
    if not "@" in dic:
        return "no dic"
    top = dic["@"]
    w1 = word_choice(top)
    w2 = word_choice(top[w1])
    ret.append(w1)
    ret.append(w2)
    while True:
        w3 = word_choice(dic[w1][w2])
        ret.append(w3)
        if w3 == ".":
            break
        w1, w2 = w2, w3
    ret = "".join(ret)
    test = {"text1" : ret}
    headers = {'accept': 'application/xml'}
    data = requests.post("http://speller.cs.pusan.ac.kr/getXMLResult", data=test, headers= headers)
    data = data.text
#     print(data)
    soup1 = BeautifulSoup(data, "html.parser")
    return soup1.select('candword')[len(soup1.select('candword'))-1].getText()

def word_choice(sel):
    keys = sel.keys()
    return random.choice(list(keys))

dict_file = "./malist/malmal.json"
if not os.path.exists(dict_file):
    fp1 = codecs.open("/Users/lesson6_mac/AtomData/python/words/class/7CM00009.txt", 'r', encoding="utf-16")
    fp2 = codecs.open('/Users/lesson6_mac/AtomData/python/words/dating/6CM00029.txt', 'r', encoding="utf-16")
    fp3 = codecs.open('/Users/lesson6_mac/AtomData/python/words/ent_news/6CM00090.txt', 'r', encoding="utf-16")
    fp4 = codecs.open('/Users/lesson6_mac/AtomData/python/words/diet/7CM00045.txt', 'r', encoding="utf-16")
    fp5 = codecs.open('/Users/lesson6_mac/AtomData/python/words/univ/6CM00023.txt', 'r', encoding="utf-16")
    fp6 = codecs.open('/Users/lesson6_mac/AtomData/python/words/traffic/4CM00014.txt', 'r', encoding="utf-16")
    fp7 = codecs.open('/Users/lesson6_mac/AtomData/python/words/travel/BRGO0347.txt', 'r', encoding="utf-16")
    soup = BeautifulSoup(fp1, "html.parser")
    text = soup.getText()
    text = text.replace("...", "")
    mec = Mecab()
    malist = mec.pos(text)
    for i in [fp2,fp3,fp4,fp5,fp6,fp7]:
        soup = BeautifulSoup(i, "html.parser")
        text = soup.getText()
        text = text.replace("...", "")
        mec = Mecab()
        for xx in mec.pos(text):
            malist.append(xx)
    words = []
    for word in malist:
        if word[0] ==".":
            words.append(word[0])
        if not word[1] in ["SC", "SSC", "SY", "SN", "SL","SSO"]:
            words.append(word[0])
    dic = make_dic(words)
    json.dump(dic, open(dict_file,"w", encoding="utf-8"))
else:
    dic = json.load(open(dict_file, "r"))


In [25]:
make_sentence(dic)

'피라미드를 세운 지 천여 개나 세웠다고 하자'

## 챗봇 만들기

In [67]:
def chat_sentence(chatchat):
    ret = []
    dict_file = "./malist/malmal.json"
    dic = json.load(open(dict_file, "r"))
    top = dic[chatchat]
    w1 = word_choice(top)
    w2 = word_choice(top[w1])
    ret.append(w1)
    ret.append(w2)
    while True:
        w3 = word_choice(dic[w1][w2])
        ret.append(w3)
        if w3 == ".":
            break
        w1, w2 = w2, w3
    ret = "".join(ret)
    test = {"text1" : ret}
    headers = {'accept': 'application/xml'}
    data = requests.post("http://speller.cs.pusan.ac.kr/getXMLResult", data=test, headers= headers)
    data = data.text
    soup1 = BeautifulSoup(data, "html.parser")
    return soup1.select('candword')[len(soup1.select('candword'))-1].getText()

In [78]:
def chatbot_start():
    print("=====================")
    print("| 1. 대화는 한글만 사용       |")
    print("| 2. 채팅종료 -> 'exit' 입력 |")
    print("=====================\n")
    print("<<<<< 대화방에 입장했어. 대화를 시작해 보자\n")
    while True:
        chatchat = input("사용자: ")
        if chatchat == "exit":
            break
#         print("강력한챗봇머신: {}".format(chat_sentence(chatchat)))
        try:
            print("강력한챗봇머신: {}".format(chat_sentence(chatchat)))
        except:
            print("강력한챗봇머신: 다른얘기를 하자 피곤하다")

In [79]:
chatbot_start()

| 1. 대화는 한글만 사용       |
| 2. 채팅종료 -> 'exit' 입력 |

<<<<< 대화방에 입장했어. 대화를 시작해 보자

사용자: 저녁
강력한챗봇머신: 먹는 과자처럼 생겼다고 나 할까요
사용자: 아침
강력한챗봇머신: 여행자가원하는장에서 바빌로니아왕국은 가장
사용자: 뭐라는거야
강력한챗봇머신: 다른얘기를 하자 피곤하다
사용자: 서운하다
강력한챗봇머신: 다른얘기를 하자 피곤하다
사용자: 그래서ㅏ
강력한챗봇머신: 다른얘기를 하자 피곤하다
사용자: 그래서
강력한챗봇머신: 평화일 것이던가요
사용자: 네
강력한챗봇머신: 맏형 노릇을 하셨더니 금방 태도를 바꾸어줍니다
사용자: 태도
강력한챗봇머신: 웬일이야 진짜 어어 근데 막 필리핀 같은 걸로
사용자: 혼자 떠들기
강력한챗봇머신: 다른얘기를 하자 피곤하다
사용자: exit
